In [1]:
from __future__ import  absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
import pandas as pd
import sys 
import os
import pickle
from collections import OrderedDict
import argparse

import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow.python.keras.layers import Lambda


import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Concatenate,Input, concatenate, Dropout
from keras.optimizers import SGD
from keras.utils import multi_gpu_model
import keras.backend as K

import matplotlib.pyplot as plt
%matplotlib inline

/home/apps/software/Python/3.6.1-IGB-gcc-4.9.4/lib/python3.6/site-packages/h5py-2.7.1-py3.6-linux-x86_64.egg/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Obejctive write (network_input, network) pairs for SA
Inputs should:
 + be contained in test set
 + be evenly distributed between high and low predicted ME inputs for networks trained on each conditions
 
Output format:
 + for High methylation: 6 dataFrames, 1 for each conditions with
 
| idx                        | p_pred | n_ume | m_me |
| ---                        | ----- | ----- | ----- |
| index in lists of input_output.pk  |   0.5  |  5      |   4   |
 + Similar for low methylation

In [2]:
#############################################
### Functions 
## Somang
def log_lkh_loss(y_true, y_pred):
    m = y_true[:,0]
    m = K.reshape(m,(-1,1))
    n = y_true[:,1]
    n = K.reshape(n,(-1,1))
    return -K.mean(m*K.log(y_pred)+n*K.log(1-y_pred),axis=-1)

def loadModel(hdf5path):
    maxlen=201
    filter_N = 80
    hd_layer_N = 40
    droprate2 = 0.2
    droprate = 0.2
    input_shape2 = (5,maxlen,1)

    inp2 = Input(shape=input_shape2)
    conv2 = Conv2D(filter_N,kernel_size=(5,6),strides=(4,1),activation='relu',padding='valid')(inp2)
    pool2 = MaxPooling2D(pool_size=(1,7),padding='valid')(conv2)
    drop2 = Dropout(droprate2)(pool2)
    flt2 = Flatten()(drop2)
    dns = Dense(hd_layer_N, activation='relu')(flt2)
    drop = Dropout(droprate)(dns)
    outp = Dense(1, activation='sigmoid')(drop)
    model = Model(inputs=inp2,outputs=outp)

    model.load_weights(hdf5path)
    sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss=log_lkh_loss,optimizer=sgd)
   
    return model

In [3]:
####################################################
## LOAD DATA 
os.chdir("../../forAlex_6conditions/")  ## added by Alex 
data = pickle.load(open("./pkls/data_pred.pk","rb"))

#input and outputs of NN
#len(inout)=7
#0:input, 1~6:outputs of each condition
inout = pickle.load(open('./pkls/input_output.pk','rb'))

#common indices for all 6 NN
#len(test_lst)=7
#0:input, 1~6:outputs of each condition
[train_indx,vald_indx,test_indx] = pickle.load(open('./pkls/test0_indx0.pk','rb'))
train_lst = [d[train_indx] for d in inout]
vald_lst = [d[vald_indx] for d in inout]
test_lst = [d[test_indx] for d in inout]

#6 conditions and 6 hdf5s
conditions = ['3A1', '3A13L', '3A2', '3A23L', '3B1', '3B13L-d1']
hdf5s = ["1110-0.8289-0.8272.hdf5", "451-1.6492-1.6407.hdf5",
         "563-1.5381-1.5321.hdf5", "243-2.6437-2.6117.hdf5", 
         "391-0.5742-0.5694.hdf5", "1059-1.0502-1.0516.hdf5"]

In [4]:
####################################################################
## Compile models
models = []
for cond , hdf5 in zip(conditions , hdf5s ):
    models.append(loadModel("./weights/"+cond+"/"+hdf5))

In [96]:
################################################
## Get highests pred_me inputs for each condiitions
n_top = 50
greatest = True
outdir= "/home/groups/song/songlab2/shared/Somang_Alex_share/interpret/MCMC/initializations"
os.makedirs(outdir, exist_ok = True)

conditionDFs = { }  # conditionName : dataFrame 
for i, (model, cond) in enumerate(zip(models, conditions)):
    preds_test = model.predict(test_lst[0],  )
    if greatest:
        tmp_sorted = np.argsort(preds_test.squeeze())[-1*(n_top):][::-1]
    else:
        tmp_sorted = np.argsort(preds_test.squeeze())[:n_top]
    test_indx_sorted = np.array([ test_indx[i] for i in tmp_sorted ] )  # indices of inputs in each list of inout
    out_true = test_lst[i+1][tmp_sorted ]   ## n_ume , n_me  for selected elements of inotu
    
    
    conditionDF = pd.DataFrame(index = test_indx_sorted,
                                data = np.concatenate([ np.asarray(preds_test[tmp_sorted]), out_true] ,axis = 1 ),
                                columns = ["p_pred" , "n_ume" , "n_me"] )
    conditionDFs[cond] = conditionDF
    
for cond , df in conditionDFs.items():
    df.to_csv( os.path.join(outdir , "{}_highest{}.tsv".format(cond , n_top)), sep = "\t" )

In [97]:
################################################
## Get lowest pred_me inputs for each condiitions
n_top = 50
greatest = False
outdir= "/home/groups/song/songlab2/shared/Somang_Alex_share/interpret/MCMC/initializations"
os.makedirs(outdir, exist_ok = True)

conditionDFs = { }  # conditionName : dataFrame 
for i, (model, cond) in enumerate(zip(models, conditions)):
    preds_test = model.predict(test_lst[0],  )
    if greatest:
        tmp_sorted = np.argsort(preds_test.squeeze())[-1*(n_top)::][::-1]
    else:
        tmp_sorted = np.argsort(preds_test.squeeze())[ :n_top]
    test_indx_sorted = np.array([ test_indx[i] for i in tmp_sorted ] )  # indices of inputs in each list of inout
    out_true = test_lst[i+1][tmp_sorted ]   ## n_ume , n_me  for selected elements of inotu
    
    
    conditionDF = pd.DataFrame(index = test_indx_sorted,
                                data = np.concatenate([ np.asarray(preds_test[tmp_sorted]), out_true] ,axis = 1 ),
                                columns = ["p_pred" , "n_ume" , "n_me"] )
    conditionDFs[cond] = conditionDF
    
for cond , df in conditionDFs.items():
    df.to_csv( os.path.join(outdir , "{}_lowest{}.tsv".format(cond , n_top)), sep = "\t" )